# Chargement du modèle GoogLeNet pré-entraîné

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

model = models.googlenet(pretrained=True)

# Gel des paramètres d'un modèle

In [6]:
for param in model.parameters():
    param.requires_grad = False

### Chargement et préparation des jeux de données avec PyTorch

In [ ]:
import torch
import torchvision
from torch.utils.data import DataLoader

train_dataset = torch.load("../data/Data_Loaders/train_dataset.pt", weights_only=False)
val_dataset   = torch.load("../data/Data_Loaders/val_dataset.pt", weights_only=False)
test_dataset  = torch.load("../data/Data_Loaders/test_dataset.pt", weights_only=False)

print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)


# Remplacement du classificateur par un réseau personnalisé

In [ ]:
num_classes = 4
# print(len(train_dataset.classes))

model.fc = nn.Sequential(
    nn.Linear(1024, 512),#nombre de fetures,nomre de layers 
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, num_classes),
)

In [10]:
# print(model)

# Configuration de l'entraînement du modèle

In [11]:
# Determiner le learning rate et loss function et aussi l’optomizer pour le modele
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# print(device)
learning_rate  = 0.001   
loss_function = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)



# Entraînement et évaluation du modèle

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model.to(device)


# Scheduler pour réduire le LR si la loss stagne (sans verbose)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
# traine model
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

    scheduler.step(epoch_loss)

#Evaluation model
def evaluate(model, loader):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = loss_function(outputs, labels)
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    return total_loss / len(loader), 100 * correct / total

train_loss, train_acc = evaluate(model, train_loader)
val_loss, val_acc     = evaluate(model, val_loader)
test_loss, test_acc   = evaluate(model, test_loader)

print(f"Train      : Loss = {train_loss:.4f}, Accuracy = {train_acc:.2f}%")
print(f"Validation : Loss = {val_loss:.4f}, Accuracy = {val_acc:.2f}%")
print(f"Test       : Loss = {test_loss:.4f}, Accuracy = {test_acc:.2f}%")



<>:105: SyntaxWarning: invalid escape sequence '\g'
<>:105: SyntaxWarning: invalid escape sequence '\g'
C:\Users\salma\AppData\Local\Temp\ipykernel_12812\858504842.py:105: SyntaxWarning: invalid escape sequence '\g'
  torch.save(model.state_dict(), "../data\googlenet_finetuned.pth")
c:\Users\salma\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\salma\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10 - Loss: 0.4583, Accuracy: 83.30%
Epoch 2/10 - Loss: 0.1815, Accuracy: 93.47%
Epoch 3/10 - Loss: 0.1648, Accuracy: 94.29%
Epoch 4/10 - Loss: 0.1593, Accuracy: 94.02%
Epoch 5/10 - Loss: 0.1273, Accuracy: 95.45%
Epoch 6/10 - Loss: 0.1322, Accuracy: 95.02%
Epoch 7/10 - Loss: 0.1046, Accuracy: 96.00%
Epoch 8/10 - Loss: 0.1191, Accuracy: 95.58%
Epoch 9/10 - Loss: 0.1208, Accuracy: 95.52%
Epoch 10/10 - Loss: 0.1038, Accuracy: 96.18%

✅ Résultats finaux :
Train      : Loss = 0.0389, Accuracy = 98.64%
Validation : Loss = 0.0294, Accuracy = 99.19%
Test       : Loss = 0.0224, Accuracy = 99.10%
✅ Modèle sauvegardé !
